In [ ]:
import math

from IPython import display
from matplotlib import cm
from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import metrics
import tensorflow as tf
from tensorflow.python.data import Dataset

tf.logging.set_verbosity(tf.logging.ERROR)
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.1f}'.format

In [ ]:
File = pd.read_csv(r'C:\Users\rohishah\Desktop\ML_Hackathon\predict_stores.csv')
File = File.fillna(0.0)
File['pop_2009'] = File['pop_2009']/1000
File['per_capita_09'] = File['per_capita_09']/1000

In [ ]:
File.head()

In [ ]:
all_features = File.columns.drop('PCH_GROC_09_14')

In [ ]:
target = ["PCH_GROC_09_14"]

In [ ]:
def preprocess_features(example_data_frame):
    selected_featur = example_data_frame[selected_features]
    processed_features = selected_featur.copy()
     # Create a synthetic feature.
    return processed_features

In [ ]:
def preprocess_targets(example_data_frame):
    output_targets = pd.DataFrame()
    output_targets[target] = example_data_frame[target] 
    return output_targets

In [ ]:
def construct_feature_columns(input_features):
    return set([tf.feature_column.numeric_column(my_feature)
              for my_feature in input_features])

In [ ]:
def my_input_fn(features, targets, batch_size=1, shuffle=True, num_epochs=None):
    # Convert pandas data into a dict of np arrays.
    features = {key:np.array(value) for key,value in dict(features).items()}                                            
 
    # Construct a dataset, and configure batching/repeating.
    ds = Dataset.from_tensor_slices((features,targets)) # warning: 2GB limit
    ds = ds.batch(batch_size).repeat(num_epochs)
    
    # Shuffle the data, if specified.
    if shuffle:
        ds = ds.shuffle(10000)
    
    # Return the next batch of data.
    features, labels = ds.make_one_shot_iterator().get_next()
    return features, labels

In [ ]:
def train_linear_classifier_model(
    learning_rate,
    steps,
    batch_size,
    training_examples,
    training_targets,
    validation_examples,
    validation_targets,
    my_optimizer,
    hidden_units):
    
    periods = 10
    steps_per_period = steps / periods
  
  # Create a linear classifier object.
    my_optimizer = my_optimizer
    my_optimizer = tf.contrib.estimator.clip_gradients_by_norm(my_optimizer, 5.0)  
    linear_classifier = tf.estimator.DNNClassifier(
      feature_columns=construct_feature_columns(training_examples),
      optimizer=my_optimizer,
     hidden_units = hidden_units)
  
  # Create input functions.
    training_input_fn = lambda: my_input_fn(training_examples, 
                                          training_targets["PCH_GROC_09_14"], 
                                          batch_size=batch_size)
    predict_training_input_fn = lambda: my_input_fn(training_examples, 
                                                  training_targets["PCH_GROC_09_14"], 
                                                  num_epochs=1, 
                                                  shuffle=False)
    predict_validation_input_fn = lambda: my_input_fn(validation_examples, 
                                                    validation_targets["PCH_GROC_09_14"], 
                                                    num_epochs=1, 
                                                    shuffle=False)
  
  # Train the model, but do so inside a loop so that we can periodically assess
  # loss metrics.
    print ("Training model...")
    training_log_losses = []
    validation_log_losses = []
    for period in range (0, periods):
        
        linear_classifier.train(input_fn=training_input_fn,
                                steps=steps_per_period)
       # Take a break and compute predictions.
        
        training_probabilities = linear_classifier.predict(input_fn=predict_training_input_fn)
        training_probabilities = np.array([item['probabilities'] for item in training_probabilities])

        validation_probabilities = linear_classifier.predict(input_fn=predict_validation_input_fn)
        validation_probabilities = np.array([item['probabilities'] for item in validation_probabilities])

        training_log_loss = metrics.log_loss(training_targets, training_probabilities)
        validation_log_loss = metrics.log_loss(validation_targets, validation_probabilities)
        # Occasionally print the current loss.
        #print(period, training_log_loss)
        # Add the loss metrics from this period to our list.
        training_log_losses.append(training_log_loss)
        validation_log_losses.append(validation_log_loss)
   
    print ("Model training finished.")
  
  # Output a graph of loss metrics over periods.
    plt.ylabel("LogLoss")
    plt.xlabel("Periods")
    plt.title("LogLoss vs. Periods")
    plt.tight_layout()
    plt.plot(training_log_losses, label="training")
    plt.plot(validation_log_losses, label="validation")
    plt.legend()

    return linear_classifier

In [ ]:
actual_df = File
actual_df.head()

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
selected_feature = []
accuracy = []
auc = []
auc_acuu_feature=[]
for outer_index in np.arange(1,36):
    for innner_index in np.arange(outer_index+1,36):
        selected_feature = all_features[outer_index:innner_index]
        training_examples, testing_examples, training_targets, testing_targets = train_test_split(actual_df[selected_feature],
                                                                       actual_df[target],test_size = 0.4,
                                                                       random_state = 1)
        linear_classifier = train_linear_classifier_model(learning_rate=0.01,
                                                            steps=500,
                                                            batch_size=10,
                                                            training_examples=training_examples,
                                                            training_targets=training_targets,
                                                            validation_examples=testing_examples,
                                                            validation_targets=testing_targets,
                                                            my_optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01),
                                                            hidden_units=[10,10])
        predict_validation_input_fn = lambda: my_input_fn(testing_examples, 
                                                          testing_targets["PCH_GROC_09_14"], 
                                                          num_epochs=1, 
                                                          shuffle=False)
        evaluation_metrics = linear_classifier.evaluate(input_fn=predict_validation_input_fn)
        #if evaluation_metrics['auc'] > 0.67 and evaluation_metrics['accuracy'] > 0.7:
        accuracy.append(evaluation_metrics['auc'])
        auc.append(evaluation_metrics['accuracy'])
        auc_acuu_feature.append(selected_feature)
        print(evaluation_metrics['auc'])
        print(evaluation_metrics['accuracy'])
        print(selected_feature)
Print("End of loop with all combination")    